In [1]:
access_token = 'EAAH24PQU4egBADGV6Xd8EnwaCmEjvlKZCLYsymsSZBVCPBvvZBtcOnMuotunZAGh4BBvAP9dHYLAnS5J70UOIXDgEbxhgHbGXOTWGAESBfSlJGj6GDZAlDvh5KYzAztgsaOxbGSlVLaT29P00gmsSLBNGXiSZBT71mRUa5rsFo2I8ZC6d4I0oBFFZCaqbyKbbCMZD'
host = '78.46.226.67'
port = '5432'
database ='postgres'
user='kirill_k'
password='swfr35$#'

address = 'postgresql://' + user +':' + password + '@' + host +':' + port + '/' + database

In [2]:
!pip install facebook-business
import psycopg2

from tqdm import tqdm

import pandas as pd
import numpy as np

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adset import AdSet
from facebook_business.exceptions import FacebookRequestError, FacebookBadObjectError
from facebook_business.adobjects.user import User
from facebook_business.api import FacebookAdsApi
from psycopg2 import OperationalError

import json

import datetime
from datetime import timedelta, datetime, date, time
import time

     |████████████████████████████████| 1.0MB 3.4MB/s 
     |████████████████████████████████| 10.1MB 9.6MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=2ce75e33ac669003b2987d987620bac7f7e6457d0f40fd65bcfe036635eea1fe
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
  Created wheel for curlify: filename=curlify-2.2.1-cp36-none-any.whl size=2660 sha256=80482f919136cfd92221aaa5f4771a21106a2f63864ec928a17f2f763afc3dbf
  Stored in directory: /root/.cache/pip/wheels/b7/5e/e5/6f90a2b21faf7b71748fcba58920ad77850c25b8f70927d418
Successfully built pycountry curlify


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
def connection_to_database(host, port, database, user, password):
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    cur = connection.cursor()
    return cur, connection

def reconnection_to_database(host, port, database, user, password, connection):
    if connection.closed > 0:
        connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
        cur = connection.cursor()
    else:
        try:
            connection.cursor().execute('SELECT 1')
        except OperationalError:
            connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
            cur = connection.cursor()
    return connection.cursor(), connection

def store_object(object_data, table):
    fields = ','.join(object_data.keys())
    values = ','.join(map(lambda x:f"%({x})s",object_data.keys()))
    query =   f"INSERT INTO {table} ({fields}) values ({values})"
#    cur.execute(query, object_data)
    connection.commit()
    
def delete_object(object_id, table):
    query_to_delete = f"""delete  from {table} where id = %s"""  
    cur.execute(query_to_delete, vars=(object_id, ))
    
def store_deleted_object(object_id, object_type, account_id, table):
    today = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    query =  f"INSERT INTO {table} values (%s, %s, %s, %s)"
    cur.execute(query, vars=(object_id, account_id, object_type,today,))
    connection.commit()

def dict2json(dictionary):
    return json.dumps(dictionary, ensure_ascii=False, sort_keys=True)

def modify_object(object_data):
    for key in object_data.keys():
        if type(object_data[key]) in (list, dict):
            object_data[key] = dict2json(object_data[key])
    object_data['recording_date'] = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    return object_data

In [4]:
print('start processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

FacebookAdsApi.init(access_token=access_token)

try:
    cur.close()
    cur, connection = connection_to_database(host, port, database, user, password)
except NameError:
    cur, connection = connection_to_database(host, port, database, user, password)

me = User(fbid='me')
accounts = list(me.get_ad_accounts())
accounts = list(map(lambda x:x.export_all_data(), accounts))
account_ids = list(map(lambda x:x['account_id'], accounts))

object_type = 'adset'
table = 'property_' + object_type + 's'

cur.execute(f""" CREATE TABLE if not exists {table}(
    id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     name VARCHAR(200), campaign JSONB,
    status VARCHAR(20),     updated_time timestamp,     start_time timestamp,
    end_time timestamp,     bid_strategy VARCHAR(50),     bid_amount BIGINT,
    bid_info JSONB,     bid_constraints JSONB,     billing_event VARCHAR(50),
    budget_remaining BIGINT,     daily_spend_cap BIGINT,     campaign_id VARCHAR(50),
    configured_status VARCHAR(20),     created_time timestamp,     daily_budget BIGINT,
    daily_min_spend_target BIGINT,     destination_type VARCHAR(20),     effective_status VARCHAR(20),
    lifetime_budget BIGINT,     lifetime_imps BIGINT,     optimization_goal VARCHAR(20),
    targeting JSONB, frequency_control_specs JSONB,   attribution_spec JSONB,     use_new_app_click BOOLEAN,
    recurring_budget_semantics BOOLEAN,     is_dynamic_creative BOOLEAN,
    source_adset JSONB,     source_adset_id VARCHAR(50),     pacing_type JSONB,
    promoted_object JSONB,     review_feedback JSONB,     recommendations JSONB, 
    instagram_actor_id VARCHAR(50),     asset_feed_id VARCHAR(50),
    adlabels JSONB,     rf_prediction_id VARCHAR(50),     adset_schedule JSONB,
    issues_info JSONB,  optimization_sub_event VARCHAR(20),
    recording_date timestamp without time zone)""")
connection.commit()

cur.execute(f""" CREATE TABLE if not exists deleted_objects(
    object_id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     
    object_type VARCHAR(20),    recording_date timestamp without time zone)""")
connection.commit()

fields = list(AdSet.Field.__dict__.keys())
exclude_list = ['__module__', '__doc__',  'daily_imps', 'frequency_cap', 'frequency_cap_reset_period', 
                'is_autobid', 'is_average_price_pacing', 'lifetime_frequency_cap', 'line_number', 
                'topline_id', 'full_funnel_exploration_mode', 'ad_keywords', 'upstream_events', 'time_stop',
               'time_start', 'rb_prediction_id', 'execution_options', 'tune_for_category', 'date_format', 'campaign_spec']

fields = list(set(fields)-set(exclude_list ))

query = """SELECT DISTINCT t.*
FROM (SELECT DISTINCT actions.object_id as adset_id, actions.account_id, 
property_adsets.recording_date is NULL as has_not_data
FROM actions LEFT JOIN property_adsets on actions.object_id = property_adsets.id
WHERE object_type = 'CAMPAIGN' and event_type not in ('receive_audience', 
'remove_shared_audience', 'create_audience', 'share_audience', 'update_audience', 'unshare_audience') and 
(property_adsets.recording_date is NULL or actions.event_time > property_adsets.recording_date)
UNION
SELECT DISTINCT insights.adset_id, insights.account_id, False as has_not_data
from insights
left join property_adsets on insights.adset_id = property_adsets.id
where property_adsets.id is null) t
LEFT JOIN deleted_objects on  t.adset_id = deleted_objects.object_id
WHERE deleted_objects.object_id IS NULL"""

adset_list = pd.read_sql(query, connection)
data = pd.read_sql(query, connection)
data = data[data['account_id'].isin(account_ids)]
adsets = dict(zip(list(data['adset_id']), data['has_not_data']))
accounts =  dict(zip(list(data['adset_id']), list(data['account_id'])))

print('start uploading \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

for adset_id in tqdm(adsets.keys()):
    try:
        adset_data = AdSet(adset_id).api_get(fields = fields).export_all_data()
        adset_data = modify_object(adset_data)
        cur, connection = reconnection_to_database(host, port, database, user, password, connection)
        if adsets[adset_id]:
            store_object(adset_data, table)
        else:
            delete_object(adset_id, table)
            store_object(adset_data, table)
        connection.commit()
        
    except FacebookRequestError as error:
        code = error._body['error']['code']
        subcode = error._body['error'].get('error_subcode', None)
        if code == 100 and (subcode == 33 or subcode == 1487221):
            cur, connection = reconnection_to_database(host, port, database, user, password, connection)
            store_deleted_object(adset_id, object_type, accounts[adset_id], 'deleted_objects')
        print('An unknown error occurred for adset ' + adset_id)

print('start refreshing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

# cur, connection = reconnection_to_database(host, port, database, user, password, connection)
# query = """REFRESH MATERIALIZED VIEW mview_adset_atributes"""
# cur.execute(query)
# connection.commit()

print('end processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

start processing 	2020-07-13 19:36:05


  0%|          | 0/397 [00:00<?, ?it/s]

start uploading 	2020-07-13 19:36:09


/usr/local/lib/python3.6/dist-packages/facebook_business/utils/api_utils.py:30: UserWarning: Value of bid_strategy is not be compatible. Expect BidStrategy; got <class 'str'>
  warnings.warn(message)
100%|██████████| 397/397 [04:22<00:00,  1.51it/s]

start refreshing 	2020-07-13 19:40:31
end processing 	2020-07-13 19:40:31
